In [47]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [48]:
# Import necessary libraries
import numpy as np
import pandas as pd
from math import sqrt
from numpy import split, array
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, RepeatVector, TimeDistributed, ConvLSTM2D
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# Load the file
path = '/content/drive/MyDrive/Pakistan_Energy_Demand/pak_load_till_2023.csv'
dataset = pd.read_csv(path, header=0)
#dataset= dataset.iloc[:,:-1]
dataset.dropna()
dataset['datetime'] = pd.to_datetime(dataset[['YEAR', 'MONTH', 'DAY', 'Hour']])
dataset.set_index('datetime', inplace=True)
dataset = dataset.drop(dataset.columns[[0 ,1, 2, 3]], axis=1)
dataset.head()


,Demand
datetime,
2019-07-01 01:00:00,19615.2656
2019-07-01 02:00:00,17842.2494
2019-07-01 03:00:00,17663.2494
2019-07-01 04:00:00,18715.0406
2019-07-01 05:00:00,18839.8286


In [49]:
import numpy as np
def split_hourly_data(hourly_load_data, num_years_train):
    # Define the number of hours in a day and days in a year
    hours_per_day = 24
    days_per_year = 365

    # Calculate the total number of hours in the data
    total_hours = len(hourly_load_data)

    # Calculate the number of hours per year
    hours_per_year = hours_per_day * days_per_year

    # Calculate the number of hours for training
    train_hours = num_years_train * hours_per_year

    # Split the data into train and test sets
    train_data_org = hourly_load_data[:-train_hours]
    test_data_org = hourly_load_data[-train_hours:]
    train_data = array(split(train_data_org, len(train_data_org)/6))
    test_data = array(split(test_data_org, len(test_data_org)/6))
    return train_data, test_data, train_data_org, test_data_org

train_data, test_data, train_data_org, test_data_org = split_hourly_data(dataset,1)
print(train_data.shape)
print(test_data.shape)
print(test_data_org.shape)


(5120, 6, 1)
(1460, 6, 1)
(8760, 1)


In [58]:


# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
    # define the end of the input sequence
      in_end = in_start + n_input
      out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end <= len(data):
      x_input = data[in_start:in_end, 0]
      x_input = x_input.reshape((len(x_input), 1))
      X.append(x_input)
      y.append(data[in_end:out_end, 0])
    # move along one time step
    in_start += 1
    return array(X), array(y)

# train the model
def build_model(train, n_steps, n_length, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 0, 20, 16
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # reshape into subsequences [samples, time steps, rows, cols, channels]
    train_x = train_x.reshape((train_x.shape[0], n_steps, 1, n_length, n_features))
    # reshape output into [samples, timesteps, features]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    model.add(Flatten())
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(100, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

# make a forecast
def forecast(model, history, n_steps, n_length, n_input):
    # flatten data
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
    # retrieve last observations for input data
    input_x = data[-n_input:, 0]
    # reshape into [samples, time steps, rows, cols, channels]
    input_x = input_x.reshape((1, n_steps, 1, n_length, 1))
    # forecast the next week
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat

# evaluate a single model
def evaluate_model(train, test, n_steps, n_length, n_input, output_csv_path):
    # fit model
    model = build_model(train, n_steps, n_length, n_input)
    # history is a list of weekly data
    history = [x for x in train]
    # walk-forward validation over each week
    predictions = list()
    for i in range(len(test)):
        # predict the week
        yhat_sequence = forecast(model, history, n_steps, n_length, n_input)
        # store the predictions
        predictions.extend(yhat_sequence)
        # get real observation and add to history for predicting the next week
        history.append(test[i, :])
    predictions = np.array(predictions)
    flat_predictions = predictions.flatten()
    # Save hourly predictions to a CSV file
    predictions_df = pd.DataFrame({'Prediction': flat_predictions})
    predictions_df.to_csv(output_csv_path, index=False)

    return output_csv_path


n_steps, n_length = 6, 6
# define the total days to use as input
n_input = n_length * n_steps
# Define the output CSV file path
output_csv_path = path

# Call evaluate_model and save hourly predictions
output_file = evaluate_model(train_data, test_data, n_steps, n_length, n_input, output_csv_path)

